In [1]:
#imports to work with...
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.data import DataLoader
import torch
import torchvision
from torchvision import transforms
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

from cl_framework.continual_learning.metrics.metric_evaluator_incdec import MetricEvaluatorIncDec
from cl_framework.utilities.matrix_logger import IncDecLogger
from torchmetrics import Recall

In [2]:
# this is used later to create the class_to_idx
def kinetics_classes(classes_csv):
    df = pd.read_csv(classes_csv)
    classes_behaviors = {}

    for _, row in df.iterrows():
        class_name = row['Class']
        subcategory = row['Subcategory']
        
        # Check if the class_name is already in the dictionary, if not, create a new entry
        if class_name not in classes_behaviors:
            classes_behaviors[class_name] = []
        
        # Add the subcategory to the corresponding class_name entry in the dictionary
        classes_behaviors[class_name].append(subcategory)

    return classes_behaviors

In [3]:
#create a mapping between classes - behaviors
folder_csv = '../Kinetics/Info/'
class_csv = os.path.join(folder_csv, 'classes.csv')
classes_behaviors = kinetics_classes(class_csv)

#create a index for each class -- {class: idx}
class_to_idx = {key: i for i, key in enumerate(classes_behaviors.keys())}


In [4]:
predictions_names = ['food', 'phone','smoking','fatigue','selfcare']
targets_names = ['target_food','target_phone','target_smoking','target_fatigue','target_selfcare']

all_behaviors_dict = {
    'food': [
        'eating burger', 'eating cake', 'eating carrots', 'eating chips', 'eating doughnuts',
        'eating hotdog', 'eating ice cream', 'eating spaghetti', 'eating watermelon',
        'sucking lolly', 'tasting beer', 'tasting food', 'tasting wine', 'sipping cup'
    ],
    'phone': [
        'texting', 'talking on cell phone', 'looking at phone'
    ],
    'smoking': [
        'smoking', 'smoking hookah', 'smoking pipe'
    ],
    'fatigue': [
        'sleeping', 'yawning', 'headbanging', 'headbutting', 'shaking head'
    ],
    'selfcare': [
        'scrubbing face', 'putting in contact lenses', 'putting on eyeliner', 'putting on foundation',
        'putting on lipstick', 'putting on mascara', 'brushing hair', 'brushing teeth', 'braiding hair',
        'combing hair', 'dyeing eyebrows', 'dyeing hair'
    ]
    }


n_task = 6
total_classes = 5

In [5]:
results_path = '../runs_trainings/no_freeze/incremental_decremental_multiclass/normal'
criterion_type = 'multiclass'
#criterion_type = 'multilabel'
seeds = [0,1,2]

In [6]:
def extract_data(task_data, metric_eval):
    predictions_list = []
    targets_list = []
    for i in range(len(predictions_names)):
        if criterion_type == 'multilabel':
            class_targets = task_data[targets_names[i]].tolist()
            targets_list.append(class_targets)
        class_predictions = task_data[predictions_names[i]].tolist()
        predictions_list.append(class_predictions)
    
    predictions = torch.Tensor(predictions_list).permute(1,0)
    if criterion_type == 'multilabel':
        targets = torch.Tensor(targets_list).permute(1,0)
    else:
        targets = torch.Tensor(task_data['target'])

    subcategory = task_data['subcategory']
    data_path = task_data['video_path']

    if criterion_type == 'multilabel':
        binarized_targets = targets
        targets = torch.Tensor(torch.argmax(targets, axis=1))
    else:
        binarized_targets = torch.Tensor(label_binarize(targets, classes=[i for i in range(5)]))

    
    metric_eval.update(targets, binarized_targets, predictions, subcategory, data_path)

In [7]:
for idx_seed in range(len(seeds)):

    seed_path = os.path.join(results_path, 'seed_' + str(seeds[idx_seed]))
        
    for exp_dir in os.listdir(seed_path):
        exp_path = os.path.join(seed_path,exp_dir)
        error_analysis_csv_path = os.path.join(exp_path,'error_analysis')

        logger_dir = os.path.join(exp_path,'logger')
        validation_logger_dir = os.path.join(exp_path,'validation_logger')

        old_logger_dir = os.path.join(logger_dir,'old')
        if not os.path.exists(old_logger_dir):
            os.mkdir(old_logger_dir)
        old_validation_logger_dir = os.path.join(validation_logger_dir,'old')
        if not os.path.exists(old_validation_logger_dir):
            os.mkdir(old_validation_logger_dir)

        # move files in old directories
        for file in os.listdir(logger_dir):
            if '.out' in file:
                os.rename(os.path.join(logger_dir,file),os.path.join(old_logger_dir,file))

        for file in os.listdir(validation_logger_dir):
            if '.out' in file:
                os.rename(os.path.join(validation_logger_dir,file),os.path.join(old_validation_logger_dir,file))


        

In [8]:
task_dict = None
for idx_seed in range(len(seeds)):

    seed_path = os.path.join(results_path, 'seed_' + str(seeds[idx_seed]))
        
    for exp_dir in os.listdir(seed_path):
        exp_path = os.path.join(seed_path,exp_dir)
        error_analysis_csv_path = os.path.join(exp_path,'error_analysis')

        
        
        logger = IncDecLogger(out_path=exp_path, n_task=n_task, task_dict=task_dict, all_behaviors_dict = all_behaviors_dict, class_to_idx= class_to_idx, num_classes=total_classes, criterion_type=criterion_type)
        for i in range(n_task):
            task_csv = error_analysis_csv_path + '/task_{}_test_error_analysis.csv'.format(str(i))
            task_data = pd.read_csv(task_csv)
            metric_eval = MetricEvaluatorIncDec('../random_tries/', num_classes=total_classes, criterion_type=criterion_type, all_behaviors_dict=all_behaviors_dict, class_to_idx=class_to_idx)
            extract_data(task_data,metric_eval)
            acc, ap, acc_per_class, mean_ap, map_weighted, precision_per_class, recall_per_class, exact_match, ap_per_subcategory, recall_per_subcategory, accuracy_per_subcategory = metric_eval.get(verbose=True)
            logger.update_accuracy(current_training_task_id=i, acc_value=acc, ap_value=ap, 
                                acc_per_class=acc_per_class, mean_ap=mean_ap, map_weighted=map_weighted, 
                                precision_per_class=precision_per_class, recall_per_class=recall_per_class, 
                                exact_match=exact_match, ap_per_subcategory=ap_per_subcategory, recall_per_subcategory=recall_per_subcategory, 
                                accuracy_per_subcategory=accuracy_per_subcategory)
            logger.update_forgetting(current_training_task_id=i)
            logger.print_latest(current_training_task_id=i)
        logger.compute_average()
        logger.print_file()


        val_logger = IncDecLogger(out_path=exp_path, n_task=n_task, task_dict=task_dict, all_behaviors_dict = all_behaviors_dict, class_to_idx= class_to_idx, num_classes=total_classes, criterion_type=criterion_type, validation_mode=True)
        for i in range(n_task):
            task_csv = error_analysis_csv_path + '/task_{}_validation_error_analysis.csv'.format(str(i))
            task_data = pd.read_csv(task_csv)
            metric_eval = MetricEvaluatorIncDec('../random_tries/', num_classes=total_classes, criterion_type=criterion_type, all_behaviors_dict=all_behaviors_dict, class_to_idx=class_to_idx)
            extract_data(task_data,metric_eval)
            acc, ap, acc_per_class, mean_ap, map_weighted, precision_per_class, recall_per_class, exact_match, ap_per_subcategory, recall_per_subcategory, accuracy_per_subcategory = metric_eval.get(verbose=True)
            val_logger.update_accuracy(current_training_task_id=i, acc_value=acc, ap_value=ap, 
                                acc_per_class=acc_per_class, mean_ap=mean_ap, map_weighted=map_weighted, 
                                precision_per_class=precision_per_class, recall_per_class=recall_per_class, 
                                exact_match=exact_match, ap_per_subcategory=ap_per_subcategory, recall_per_subcategory=recall_per_subcategory, 
                                accuracy_per_subcategory=accuracy_per_subcategory)
            val_logger.update_forgetting(current_training_task_id=i)
            val_logger.print_latest(current_training_task_id=i)
        val_logger.compute_average()
        val_logger.print_file()

 - task accuracy: 0.42702702702702705
 - task average precision: tensor([0.5965, 0.1740, 0.1094, 0.2291, 0.7072])
 - task acc per class: [0.6783783783783783, 0.9054054054054054, 0.7297297297297297, 0.8108108108108109, 0.7297297297297297]
 - task precision per class: tensor([0.6061, 0.2222, 0.1196, 0.2222, 0.5746])
 - task recall per class: tensor([0.4286, 0.0667, 0.3667, 0.1600, 0.6417])
 - task mAP: 0.36324262619018555
 - task weighted mAP: 0.5090140700340271

 >>> Test on task  0 : acc= 42.7%,  |  <<<
 - task accuracy: 0.47297297297297297
 - task average precision: tensor([0.5697, 0.1759, 0.1306, 0.2374, 0.7130])
 - task acc per class: [0.6621621621621622, 0.9108108108108108, 0.9027027027027027, 0.7378378378378379, 0.7324324324324324]
 - task precision per class: tensor([0.5641, 0.2857, 0.1250, 0.2418, 0.5714])
 - task recall per class: tensor([0.4714, 0.0667, 0.0333, 0.4400, 0.7000])
 - task mAP: 0.36530667543411255
 - task weighted mAP: 0.5037170648574829

 >>> Test on task  1 : ac

 - task accuracy: 0.505
 - task average precision: tensor([0.6346, 0.1064, 0.2959, 0.3758, 0.6495])
 - task acc per class: [0.7, 0.88, 0.875, 0.835, 0.72]
 - task precision per class: tensor([0.5568, 0.0000, 0.3684, 0.4400, 0.5312])
 - task recall per class: tensor([0.7000, 0.0000, 0.3500, 0.3667, 0.5667])
 - task mAP: 0.4124302268028259
 - task weighted mAP: 0.5135602951049805

 >>> Test on task  4 : acc= 50.5%,  |  <<<
 - task accuracy: 0.565
 - task average precision: tensor([0.7216, 0.1721, 0.2754, 0.3373, 0.7098])
 - task acc per class: [0.78, 0.89, 0.86, 0.84, 0.76]
 - task precision per class: tensor([0.6548, 0.2500, 0.3182, 0.4545, 0.5882])
 - task recall per class: tensor([0.7857, 0.0500, 0.3500, 0.3333, 0.6667])
 - task mAP: 0.44324928522109985
 - task weighted mAP: 0.5608522891998291

 >>> Test on task  5 : acc= 56.5%,  |  <<<
